In [21]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

Внимательно изучив наши данные, мы поняли, что можно попробовать обучить  модель, которая будет предсказывать заработную плату на основе следующих признаков: опыт, занятость, рейтинг компании, формат работы

Для начала снова взглянем на данные, чтобы преобразовать их для последующего обучения.

In [36]:
df = pd.read_csv('hh_clean.csv')
df.head()

,link,vacancy,company,city,experience,company_rating,working_hours,employee_location,wage_k
0,https://adsrv.hh.ru/click?b=1003372&place=35&m...,Водитель со своим грузовым автомобилем,ИП Петрикин Александр Вячеславович,Москва,0,4.5,Полная занятость,В офисе,180.0
1,https://adsrv.hh.ru/click?b=993174&place=36&me...,Консультант поддержки в Яндекс Путешествия,Яндекс Крауд: Поддержка,Москва,0,4.0,Полная занятость,Удалённая работа,30.0
2,https://hh.ru/vacancy/98672015?hhtmFrom=vacanc...,Менеджер по заполнению анкет (удаленно),ИП Кабин Дмитрий Владимирович,Москва,0,3.5,Полная занятость,Удалённая работа,36.8
3,https://hh.ru/vacancy/98657055?hhtmFrom=vacanc...,Ручной тестировщик (QA Manual Junior),Oxytocin,Москва,0,3.5,Полная занятость,В офисе,60.0
4,https://hh.ru/vacancy/93848299?utm_source=head...,Торговый представитель (Водитель-курьер),ЦУМ,Москва,1,4.0,Полная занятость,Сменный график,170.0


1. `Часы работы` это категориальный бинарный признак, содержащий значения "Полная занятость" и "Частичная занятость" - идеально для применения OHE

In [37]:
ohe = OneHotEncoder()
data1 = ohe.fit_transform(pd.DataFrame(df['working_hours']))
working_hours = pd.DataFrame(data1.toarray(), columns=['full_time', 'part_time'], dtype=int)
working_hours

,full_time,part_time
0,1,0
1,1,0
2,1,0
3,1,0
4,1,0
...,...,...
2072,0,1
2073,0,1
2074,0,1
2075,0,1


Теперь вместо столбца `working_hours` будем использовать новый признак - `full_time` - принимающий значения "1" - полная занятость и "0" - частичная занятость

In [38]:
df['full_time'] = working_hours['full_time']
del df['working_hours']
df.head()

,link,vacancy,company,city,experience,company_rating,employee_location,wage_k,full_time
0,https://adsrv.hh.ru/click?b=1003372&place=35&m...,Водитель со своим грузовым автомобилем,ИП Петрикин Александр Вячеславович,Москва,0,4.5,В офисе,180.0,1
1,https://adsrv.hh.ru/click?b=993174&place=36&me...,Консультант поддержки в Яндекс Путешествия,Яндекс Крауд: Поддержка,Москва,0,4.0,Удалённая работа,30.0,1
2,https://hh.ru/vacancy/98672015?hhtmFrom=vacanc...,Менеджер по заполнению анкет (удаленно),ИП Кабин Дмитрий Владимирович,Москва,0,3.5,Удалённая работа,36.8,1
3,https://hh.ru/vacancy/98657055?hhtmFrom=vacanc...,Ручной тестировщик (QA Manual Junior),Oxytocin,Москва,0,3.5,В офисе,60.0,1
4,https://hh.ru/vacancy/93848299?utm_source=head...,Торговый представитель (Водитель-курьер),ЦУМ,Москва,1,4.0,Сменный график,170.0,1


2. `Формат работы` - не самый приятный признак, но мы придумали, как его преобразовать: оставим столбцы "офис" и "удаленная работа" как категориальные бинарные признаки.

In [39]:
df['employee_location'].unique()

array(['В офисе', 'Удалённая работа', 'Сменный график', 'Вахтовый метод',
       'Гибкий график'], dtype=object)

In [40]:
data2 = ohe.fit_transform(pd.DataFrame(df['employee_location']))
employee_location = pd.DataFrame(data2.toarray(), columns=['office', 'distant', 'shift_work', 'vakhta', 'flexible_schedule'], dtype=int)
employee_location

,office,distant,shift_work,vakhta,flexible_schedule
0,1,0,0,0,0
1,0,0,0,0,1
2,0,0,0,0,1
3,1,0,0,0,0
4,0,0,0,1,0
...,...,...,...,...,...
2072,1,0,0,0,0
2073,1,0,0,0,0
2074,1,0,0,0,0
2075,1,0,0,0,0


Чтобы не грузить таблицу лишними столбцами, ставим только `office` и `distant` со значениями "1" - да, "0" - нет; комбинация "0", "0" этих признаков будет означать другие варианты работы - Сменный/Гибкий график или Вахтовый метод

In [41]:
df[['office', 'distant']] = employee_location[['office', 'distant']]
del df['employee_location']
df.head()

,link,vacancy,company,city,experience,company_rating,wage_k,full_time,office,distant
0,https://adsrv.hh.ru/click?b=1003372&place=35&m...,Водитель со своим грузовым автомобилем,ИП Петрикин Александр Вячеславович,Москва,0,4.5,180.0,1,1,0
1,https://adsrv.hh.ru/click?b=993174&place=36&me...,Консультант поддержки в Яндекс Путешествия,Яндекс Крауд: Поддержка,Москва,0,4.0,30.0,1,0,0
2,https://hh.ru/vacancy/98672015?hhtmFrom=vacanc...,Менеджер по заполнению анкет (удаленно),ИП Кабин Дмитрий Владимирович,Москва,0,3.5,36.8,1,0,0
3,https://hh.ru/vacancy/98657055?hhtmFrom=vacanc...,Ручной тестировщик (QA Manual Junior),Oxytocin,Москва,0,3.5,60.0,1,1,0
4,https://hh.ru/vacancy/93848299?utm_source=head...,Торговый представитель (Водитель-курьер),ЦУМ,Москва,1,4.0,170.0,1,0,0


In [42]:
df.to_csv('hh_OHE.csv', index=False)